https://towardsdatascience.com/evaluating-classification-models-with-kolmogorov-smirnov-ks-test-e211025f5573#:~:text=The%20KS%20statistic%20for%20two,another%20metric%20to%20evaluate%20classifiers.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from xgboost import XGBClassifier

In [2]:
data = pd.read_csv("../../10_fold_cross_validation/train_10folds.csv")
data = data.drop(['Kfold'], axis=1)
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET
0,0.335201,1.714286,-2.072068,5.975550,0.000000,12.132734,9.551078,0.000000,38.974594,43.638476,6,0,1,1.0
1,0.226791,2.000000,-2.362642,7.150190,5.948339,0.000000,9.967957,0.000000,10.440599,11.336786,6,0,0,1.0
2,0.211302,1.772727,-2.089721,7.912349,9.837253,29.297126,4.983979,9.837253,10.902925,24.265468,4,0,1,0.0
3,0.226898,1.869565,-2.357502,6.433493,5.948339,35.334614,15.284746,0.000000,11.126903,12.263211,5,0,0,0.0
4,0.158370,1.870968,-2.421374,7.991366,0.000000,23.362825,19.935914,6.420822,0.000000,18.460054,9,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.152613,2.000000,-2.415835,6.432407,0.000000,35.334614,9.967957,11.343745,11.257379,6.066367,6,0,0,1.0
2735,0.335203,1.700000,-2.001320,6.164147,0.000000,30.331835,4.983979,0.000000,10.902925,36.528679,2,0,1,0.0
2736,0.264178,1.833333,-2.414068,5.737009,5.948339,38.112943,9.551078,11.784535,11.126903,11.614772,7,0,0,1.0
2737,0.338995,1.815789,-2.065301,7.218862,5.907180,59.014740,24.544948,18.386966,43.634305,6.196844,9,0,2,0.0


In [3]:
X, y = data.drop(columns = ["TARGET"]), data["TARGET"]

In [4]:
# Create the model object
model = XGBClassifier()
# Fit the model to the training data
model.fit(X, y)
# Predict the classes on the test data
y_pred = model.predict(X)
# Predict the classes on the test data, and return the probabilities for each class
y_proba = model.predict_proba(X)

In [5]:
def cdf(sample, x, sort = False):
    '''
    Return the value of the Cumulative Distribution Function, evaluated for a given sample and a value x.
    
    Args:
        sample: The list or array of observations.
        x: The value for which the numerical cdf is evaluated.
    
    Returns:
        cdf = CDF_{sample}(x)
    '''
    
    # Sorts the sample, if needed
    if sort:
        sample.sort()
    
    # Counts how many observations are below x
    cdf = sum(sample <= x)
    
    # Divides by the total number of observations
    cdf = cdf / len(sample)
    
    return cdf

In [6]:
# Gets the class CDFs
def get_classes_cdf(y_real, y_proba):
    # Unite both visions to be able to filter
    df = pd.DataFrame()
    df['real'] = y_real
    df['proba'] = y_proba[:, 1]
    
    # Recover each class
    class0 = df[df['real'] == 0].sort_values('proba')
    class1 = df[df['real'] == 1].sort_values('proba')
    
    # Calculates the cdfs
    cdf0 = np.array([cdf(class0['proba'].values, x, sort = False) for x in class0['proba'].values])
    cdf1 = np.array([cdf(class1['proba'].values, x, sort = False) for x in class1['proba'].values])
    
    # Results
    results =  {
        'cdf0': cdf0,
        'cdf1': cdf1,
        'proba0': class0['proba'].values, 
        'proba1': class1['proba'].values, 
    }
    
    return results

In [7]:
cdf_good = get_classes_cdf(y_pred, y_proba)

In [9]:
plt.figure(figsize = (10, 10), dpi=1200)

plt.title("XGB Classifier KS Statistic")
sns.lineplot(x = cdf_good['proba0'], y = cdf_good['cdf0'], color = 'b', linewidth = 3)
sns.lineplot(x = cdf_good['proba1'], y = cdf_good['cdf1'], color = 'g', linewidth = 3)
plt.legend(["class 0", "class 1"])
plt.rcParams.update({'font.size': 18})
plt.tight_layout()
plt.savefig('..\images\KS_Statistics.png')